In [2]:
import pickle
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
vis = []

# Load data
sim = pd.compat.pickle_compat.load(open('sim', 'rb'))
movies = pd.compat.pickle_compat.load(open('movies_list', 'rb'))
movies_df = movies

# Create a TF-IDF vectorizer for tags
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['tags'])

def recommend(user_choice, user_history):
    if user_choice in movies_df['title'].values:
        # User choice matches a movie title, recommend based on the movie title
        ind = movies_df[movies_df['title'] == user_choice].index[0]
        recom_list = sorted(list(enumerate(sim[ind])), key=lambda x: x[1], reverse=True)
    else:
        # User choice doesn't match any movie title, recommend based on similarity to tags
        tfidf_user_choice = tfidf_vectorizer.transform([user_choice])
        cosine_sim = cosine_similarity(tfidf_user_choice, tfidf_matrix)
        recom_list = sorted(list(enumerate(cosine_sim[0])), key=lambda x: x[1], reverse=True)

    count = 0
    upd_recom_list = []
    for i in recom_list:
        if count >= 10:
            break
        if i not in vis:
            vis.append(i)
            upd_recom_list.append(i)
            count += 1

    recommended_movies = []
    recommended_ids = []
    recommended_popularity = []
    recommended_votecount = []

    for i in upd_recom_list:
        movie_id = movies_df.iloc[i[0]].id
        pop = movies_df.iloc[i[0]].popularity
        vote_c = movies_df.iloc[i[0]].vote_count
        recommended_ids.append(movie_id)
        recommended_movies.append(movies_df.iloc[i[0]].title)
        recommended_popularity.append(pop)
        recommended_votecount.append(vote_c)

    return recommended_movies, recommended_ids, recommended_popularity, recommended_votecount

# Example usage
user_choice = "give me movies with animation and family"
user_history = []  # You can add user's previous choices if needed

recommended_movies, recommended_ids, recommended_popularity, recommended_votecount = recommend(user_choice, user_history)
for ind in range(len(recommended_movies)):
    print(recommended_movies[ind], recommended_ids[ind], recommended_popularity[ind], recommended_votecount[ind])


Fantasia 2000 49948 14.530946 292
Grindhouse 285923 16.637642 459
Tupac: Resurrection 21525 1.314576 26
Scary Movie 2 4248 35.376971 982
Darkness 11056 6.340135 112
The Maid's Room 278348 1.501688 4
Finishing The Game 13983 0.459625 8
Orgazmo 8675 5.06516 115
Blow Out 11644 13.628601 236
The Addams Family 2907 37.179547 848


In [5]:
import pandas as pd
print(pd.__version__)


2.0.3


In [7]:
import sys
sys.version


'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import re
import pandas as pd
import requests
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

app = Flask(__name__)
CORS(app)  # Add this line to enable CORS for all routes

api_key = '5b923f780b5401e92b81a39fb35f266d'


vis = []

# Load data
sim = pd.compat.pickle_compat.load(open('sim', 'rb'))
movies = pd.compat.pickle_compat.load(open('movies_list', 'rb'))
movies_df = movies

# Create a TF-IDF vectorizer for tags
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['tags'])


def recommend(user_choice, user_history):
    if user_choice in movies_df['title'].values:
        # User choice matches a movie title, recommend based on the movie title
        ind = movies_df[movies_df['title'] == user_choice].index[0]
        recom_list = sorted(
            list(enumerate(sim[ind])), key=lambda x: x[1], reverse=True)
    else:
        # User choice doesn't match any movie title, recommend based on similarity to tags
        tfidf_user_choice = tfidf_vectorizer.transform([user_choice])
        cosine_sim = cosine_similarity(tfidf_user_choice, tfidf_matrix)
        recom_list = sorted(
            list(enumerate(cosine_sim[0])), key=lambda x: x[1], reverse=True)

    count = 0
    upd_recom_list = []
    for i in recom_list:
        if count >= 20:
            break
        if i not in vis:
            vis.append(i)
            upd_recom_list.append(i)
            count += 1

    recommended_movies = []
    recommended_ids = []
    recommended_popularity = []
    recommended_votecount = []

    for i in upd_recom_list:
        movie_id = movies_df.iloc[i[0]].id
        pop = movies_df.iloc[i[0]].popularity
        vote_c = movies_df.iloc[i[0]].vote_count
        recommended_ids.append(movie_id)
        recommended_movies.append(movies_df.iloc[i[0]].title)
        recommended_popularity.append(pop)
        recommended_votecount.append(vote_c)

    return recommended_movies, recommended_ids, recommended_popularity, recommended_votecount


@app.route("/")
def front():
    return render_template('index.html')


@app.route("/search-query", methods=["POST"])
def recommend_movies():
    data = request.get_json()  # Get JSON data from the request
    user_query = data.get("searchText")


# Example usage
    # user_choice = "give me movies with animation and family"
    user_history = []  # You can add user's previous choices if needed

    recommended_movies, recommended_ids, recommended_popularity, recommended_votecount = recommend(
        user_query, user_history)
    # for ind in range(len(recommended_movies)):
    #     print(recommended_movies[ind], recommended_ids[ind],
    #         recommended_popularity[ind], recommended_votecount[ind])

    # top_10_popular_movies = matching_movies.head(6)

    # Prepare the response in JSON format
    response_data = {
        "RecommendedMovies": []
    }
#     print(recommend_movies)
    for i in range(len(recommended_movies)):
        movie_id = int(recommended_ids[i])
        movie_name = recommended_movies[i]
        popularity = recommended_popularity[i]
        vote_count = int(recommended_votecount[i])

        poster_url = get_movie_poster_url(movie_id)
        movie_info = {
            "MovieName": movie_name,
            "MovieID": movie_id,
            "Popularity": popularity,
            "VoteCount": vote_count,
            "PosterURL": poster_url
        }
        response_data["RecommendedMovies"].append(movie_info)

    return jsonify(response_data)


def get_movie_poster_url(movie_id):
    response = requests.get(
        f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}')
    if response.status_code == 200:
        poster = response.json()
        poster_path = poster.get('poster_path')
        if poster_path:
            return 'https://image.tmdb.org/t/p/w500/' + poster_path
    return None


if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2023 03:20:15] "OPTIONS /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:20:24] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:20:44] "OPTIONS /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:20:57] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:20:59] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:00] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:18] "OPTIONS /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:25] "OPTIONS /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:31] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:35] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:21:37] "POST /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:22:51] "OPTIONS /search-query HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 03:23:03] "POST /search-q

In [10]:
!pip install Flask-CORS


  Using cached Flask_Cors-4.0.0-py2.py3-none-any.whl.metadata (5.4 kB)
Using cached Flask_Cors-4.0.0-py2.py3-none-any.whl (14 kB)
